# <center >Accidents corporels de la route
### Ouverture de la base de données

Faire les importations nécessaires :

In [1]:
#pip install graphviz

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import graphviz
import seaborn as sns

Importer les 4 fichiers :

In [5]:
df_carac = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/e22ba475-45a3-46ac-a0f7-9ca9ed1e283a", sep = ";")
df_lieux = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/2ad65965-36a1-4452-9c08-61a6c874e3e6", sep = ";")
df_usagers = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/36b1b7b3-84b4-4901-9163-59ae8a9e3028", sep = ";")
df_vehicules = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/780cd335-5048-4bd6-a841-105b44eb2667", sep = ";")

Une première sélection a été réalisée. Nous avons éliminé les variables qui ne semblaient avoir aucun lien avec la gravité de l'accident.

In [6]:
df_carac = df_carac[["Num_Acc", "jour", "mois", "an", "hrmn", "lum", "dep", "com", "agg", "int", "atm", "col", "lat", "long"]]
df_lieux = df_lieux[["Num_Acc", "catr", "circ", "nbv", "surf", "prof", "plan", "infra", "situ", "vma"]] 
df_usagers = df_usagers[["Num_Acc", "id_vehicule", "place", "catu", "sexe", "grav", "an_nais", "trajet", "secu1"]]
df_vehicules = df_vehicules[["Num_Acc", "id_vehicule", "catv", "obs", "obsm", "choc", "occutc"]]

Fusionner les 4 bases de données :

In [13]:
df = df_carac.merge(df_lieux, how='left', on=['Num_Acc'])
df = df.merge(df_vehicules, how='inner', on=['Num_Acc'])
df = df.merge(df_usagers, how='inner', on=['Num_Acc', 'id_vehicule'])

Au total, en 2019, il y eu 58 840 accidents corporels de route qui ont concernés au total 100 710 véhicules et 132 977 personnes. \
Ainsi, dans notre table globale, nous avons 132 977 observations et 36 variables.

A l'aide du descriptif des modalités, nous remplaçons les modalités par leur nom complet pour une meilleure compréhension.

In [14]:
# Variables de la base Caracteristiques
df["lum"]= df['lum'].map({1:'Plein jour',
                        2:'Crépuscule ou aube',
                        3: "Nuit sans éclairage public",
                        4: "Nuit avec éclairage public non allumé",
                        5: "Nuit avec éclairage public allumé"})
df['agglo'] = df['agg'].map({1:'Hors agglomeration',
                             2:'Agglomeration'})
del df['agg']


df['int']=df['int'].map({1:'Hors intersection',
                        2:'Intersection en X',
                        3:'Intersection en T',
                        4:'Intersection en Y',
                        5:'Intersection à plus de 4 branches',
                        6:'Giratoire',
                        7:"Place",
                        8:'Passage à niveaux',
                        9:'Autre'})
df['atm'] = df['atm'].map({1:'Normale',
                             2:'Pluie legere',
                             3:'Pluie forte',
                             4:'Neige',
                          5:'Brouillard',
                          6:'Vent fort',
                          7:'Eblouissant',
                          8:'Couvert',
                          9:'Autre'})
df['collision'] = df['col'].map({1:'2 - frontale',
                             2:'2-arriere',
                             3:'2-cote',
                             4:'3-chaine',
                          5:'3-multiple',
                          6:'autre',
                          7:'Sans collision'})
del df['col']


# Variables de la base Usagers
df['catu'] = df['catu'].map({1:'Conducteur',
                             2:'Passager',
                             3:'Pieton'})
df['sexe'] = df['sexe'].map({1:'Homme',
                             2:'Femme'})
df['grav'] = df['grav'].map({1:'Indemne',
                             2:'Tue',
                             3:'Blesse hospitalise',
                             4:'Blesse leger'})


# Variables de la base Lieux
df['catr'] = df['catr'].map({1:'Autoroute',
                             2:'Route nationale',
                             3:'Route départementale',
                             4:'Voie Communale',
                          5:'Hors réseau public',
                          6:'Parc de stationnement',
                          7:'Routes de métropole urbaine',
                          9:'Autre'})
df["circ"]=df["circ"].map({1:"A sens unique",
                          2:"Bidirectionnelle",
                          3:'A chaussées séparées',
                          4:"Avec voies d'affectation variable"})
df["prof"]=df["prof"].map({1:"Plat",
                          2:"Pente",
                          3:'Sommet de côte',
                          4:"Bas de côte"})
df["plan"]=df["plan"].map({1:"Rectiligne",
                          2:"En courbe à gauche",
                          3:'En courbe à droite',
                          4:"En S"})
df['surf'] = df['surf'].map({1:'Normale',
                             2:'Mouillée',
                             3:'Flaques',
                             4:'Inondée',
                          5:'Enneigée',
                          6:'Boue',
                          7:'Verglecée',
                             8:'Corps gras - huile',
                          9:'Autre'})
df['situ'] = df['situ'].map({1:'Sur chaussée',
                             2:"Sur bande d'arrêt d'urgence",
                             3:'Sur accotement',
                             4:'Sur trottoir',
                          5:'Sur piste cyclable',
                          6:'Sur autre voie spéciale',
                          8:'Autre'})

# Création de nouvelles variables
df['age'] = 2019 - df['an_nais']

### Appropriation du jeu de données et structure de la base

In [8]:
df.columns

Index(['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'int',
       'atm', 'lat', 'long', 'catr', 'circ', 'nbv', 'surf', 'prof', 'plan',
       'infra', 'situ', 'vma', 'id_vehicule', 'catv', 'obs', 'obsm', 'choc',
       'occutc', 'place', 'catu', 'sexe', 'grav', 'an_nais', 'trajet', 'secu1',
       'agglo', 'collision', 'age'],
      dtype='object')

In [9]:
df.head(5)

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,int,atm,...,place,catu,sexe,grav,an_nais,trajet,secu1,agglo,collision,age
0,201900000001,30,11,2019,01:30,Nuit avec éclairage public non allumé,93,93053,Hors intersection,Normale,...,2.0,Passager,Femme,Blesse leger,2002.0,0.0,1.0,Hors agglomeration,2-arriere,17.0
1,201900000001,30,11,2019,01:30,Nuit avec éclairage public non allumé,93,93053,Hors intersection,Normale,...,1.0,Conducteur,Femme,Blesse leger,1993.0,5.0,1.0,Hors agglomeration,2-arriere,26.0
2,201900000001,30,11,2019,01:30,Nuit avec éclairage public non allumé,93,93053,Hors intersection,Normale,...,1.0,Conducteur,Homme,Indemne,1959.0,0.0,1.0,Hors agglomeration,2-arriere,60.0
3,201900000002,30,11,2019,02:50,Nuit sans éclairage public,93,93066,Hors intersection,Normale,...,1.0,Conducteur,Femme,Blesse leger,1994.0,0.0,1.0,Hors agglomeration,autre,25.0
4,201900000003,28,11,2019,15:15,Plein jour,92,92036,Hors intersection,Normale,...,1.0,Conducteur,Homme,Indemne,1996.0,0.0,1.0,Hors agglomeration,3-chaine,23.0


In [1]:
#df.dtypes

In [18]:
#print(df.describe(include='all'))

### Etude des valeurs manquantes :

Dans ce jeu de données, les valeurs non renseignées sont indiquées par "-1". \
La majorité des variables ne possèdent pas de valeurs manquantes. \
Quelques variables possèdent moins d'1% de valeurs manquantes (ex : collision, état de la surface). \
Concernant la variable "Circulation", il y a 5.06% sont des valeurs manquantes. 

In [12]:
colonne = ["lum", "agglo", "int", "collision", "atm", "catr", "circ", "surf", "prof", "plan", "infra", "situ",
           "catu", "sexe", "choc", "grav"]
def effecitf_val_manquantes():
    for elem in colonne :
        eff = (df[elem].isna().sum()).round(2)
        freq = ((df[elem].isna().sum())/len(df[elem])*100).round(2)
        print("Pour la variable " + elem + " :  " + freq.astype(str) + " % \n ce qui représente un effectif de :    " + eff.astype(str))
        print("\n")
#effecitf_val_manquantes()

### Etude des modalités rares :

In [19]:
colonne = ["lum", "agglo", "int", "collision", "atm", "catr", "circ", "surf", "prof", "plan", "infra", "situ",
           "catu", "sexe", "choc", "grav"]
def effectif_pourcentage():
    for elem in colonne :
        print("Pour la variable " + elem + " :")
        print((df[elem].value_counts())/len(df[elem])*100)
        print("\n")
#effectif_pourcentage()

En statistique, il n'est pas souhaitable d'avoir des modalités rares qui ont un effectif inférieur à 5%. Pour cela, nous avons regroupé les modalités entre elles en suivant deux méthodes différentes. Dans un premier temps, nous avons regroupé des modalités entre elles lorsqu'elles avaient un sens proche. Par exemple, "Intersection en X" et "Intersection en Y" correspondent à la modalité commune "Intersection".

In [33]:
# La modalité "Avec éclairage non allumé" devient "Sans éclairage"
df['lum']=df['lum'].replace(["Nuit avec éclairage public non allumé"], "Nuit sans éclairage public")

# Variable intersection : on regroupe tous les types d'intersections
df['int']=df['int'].replace(["Intersection en X", "Intersection en Y", "Intersection en T"], "Intersection")

# Variable catr :


# Variable circulation : 
df['circ']=df['circ'].replace(["Avec voies d'affectation variable"], "Bidirectionnelle")

# Prof :
df['prof'] = df['prof'].replace({"Bas de côte" : "Sommet de côte"})

# Plan : 
df['plan'] = df['plan'].replace(["En courbe à gauche", "En courbe à droite", "En S"], "Courbe")

#Infra :
df['infra'] = df['infra'].replace([0, 5], ["Aucun", "Carrefour aménagé"])

La seconde méthode est de regrouper les modalités ayant représentant moins de 5% entre elles afin de créer une modalité "Autre".

In [35]:
colonne = ["lum", "agglo", "sexe", "int", "collision", "atm", "surf", "circ"]
def regroupement_modalites():
    for elem in colonne :
        frequence = df[elem].value_counts()/len(df[elem])*100
        for i in range(len(frequence)) :
            if frequence[i] < 5 :
                df.loc[df[elem]==frequence.index[i], elem] = "Autre"   
regroupement_modalites()
#effectif_pourcentage()

## Ajout de variables 

In [21]:
conditions_obstacle = [
                        (df['obs']>0) & (df['obsm']>0),
                        (df['obs']>0) & (df['obsm']<=0),
                        (df['obs']<=0) & (df['obsm']>0),
                        (df['obs']==0) & (df['obsm']==0)]

types_obstacle = ['1 obstacle mobile et 1 obstacle fixe', '1 obstacle fixe', '1 obstacle mobile', 'Pas dobstacle']

df['obstacle'] = np.select(conditions_obstacle, types_obstacle, default='-1')

On crée ensuite une nouvelle variable de catégorie d'âge, liée aux usagers, qu'on appelle 'age', à partir de la variable existante 'an_nais' (année de naissance)

In [ ]:
conditions_age = [
                    (0 <= 2019 - df['an_nais']) & (2019 - df['an_nais'] <= 15),
                    (16 <= 2019 - df['an_nais']) & (2019 - df['an_nais'] <= 30),
                    (31 <= 2019 - df['an_nais']) & (2019 - df['an_nais'] <= 65),
                    (66 <= 2019 - df['an_nais'])]

categories_age = ['enfant', 'jeune', 'adulte', 'personne agee']

df['age'] = np.select(conditions_age, categories_age, default='-1')